In [1]:
import json
import configparser
import requests
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import pyspedas.mms as mms
from functions_1 import julian
from sgp4.api import Satrec
import time

r_e = 6371.

config = configparser.ConfigParser()
config.read("../SpTrack-2.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query"

In [34]:
a = pd.read_pickle('../WHISPER/whi_int.pkl')
peri_dt_list = (a['dt2'] - a['dt1'])/2 + a['dt1']

In [7]:
def tle_get(epoch_start, days_no, launch_dt:str, decay_dt:str, periapsis): 
    epoch_range = []
    for j in range(days_no + 1):
        epoch_range.append((epoch_start + timedelta(days=j)).strftime('%Y-%m-%d'))

    with requests.Session() as session:

        resp = session.post(uriBase + requestLogin, data = siteCred)
        if resp.status_code != 200:
            raise MyError(resp, "POST fail on login")

        satIds = []
        epoch = []
        tle_1 = []
        tle_2 = []
        
        count=1
        
        for i in range(len(epoch_range)-1):
            if count > 10:
                print("Snoozing for 90 secs for rate limit reasons (max 30/min and 300/hr)...")
                time.sleep(90)
                count = 1
            
            epoch_s = epoch_range[i]
            epoch_e = epoch_range[i+1]

            requestTLE_1 = "/class/gp_history/LAUNCH_DATE/<" + launch_dt + "/DECAY_DATE/null-val/OBJECT_TYPE/PAYLOAD/PERIAPSIS/<" + str(periapsis) + "/EPOCH/" + epoch_s + "--" + epoch_e + "/predicates/NORAD_CAT_ID,EPOCH,TLE_LINE1,TLE_LINE2/format/json"
            resp = session.get(uriBase + requestCmdAction + requestTLE_1)
            if resp.status_code != 200:
                print(resp)
                raise MyError(resp, "GET fail on request for satellites")
            
            retData = json.loads(resp.text)
            for e in retData:
                satIds.append(e['NORAD_CAT_ID'])
                tle_1.append([e['TLE_LINE1']])
                tle_2.append([e['TLE_LINE2']])
                epoch.append(e['EPOCH'])
            
            requestTLE_2 = "/class/gp_history/LAUNCH_DATE/<" + launch_dt + "/DECAY_DATE/>" + decay_dt + "/OBJECT_TYPE/PAYLOAD/PERIAPSIS/<" + str(periapsis) + "/EPOCH/" + epoch_s + "--" + epoch_e + "/predicates/NORAD_CAT_ID,EPOCH,TLE_LINE1,TLE_LINE2/format/json"
            resp = session.get(uriBase + requestCmdAction + requestTLE_1)
            if resp.status_code != 200:
                print(resp)
                raise MyError(resp, "GET fail on request for satellites")
            
            retData = json.loads(resp.text)
            for e in retData:
                satIds.append(e['NORAD_CAT_ID'])
                tle_1.append([e['TLE_LINE1']])
                tle_2.append([e['TLE_LINE2']])
                epoch.append(e['EPOCH'])
            
            count+=1
            
    a = [datetime.strptime(i, '%Y-%m-%dT%H:%M:%S.%f') for i in epoch]
    tle_df = pd.DataFrame({'id': satIds, 'epoch': a, 'tle_1': tle_1, 'tle_2': tle_2})

    session.close()
    
    return tle_df

def find_conj(dt_start, days_no, obs, min_buffer, is_dist_buffer, peri_dt_list):
    tle_df = pd.read_pickle('tle_' + str(dt_start.year) + '_' + ('%02d' % dt_start.month) + '.pkl')
    
    is_mindist_list = []
    is_mindist_t = []
    is_mindist_id = []
    is_mindist_alt = []
    
    peri_dt_trun = peri_dt_list[peri_dt_list > dt_start][peri_dt_list < (dt_start + timedelta(days=days_no-1))]
    
    for i in peri_dt_trun:
        j_sta = julian(i - timedelta(minutes=min_buffer))[0]
        j_end = julian(i + timedelta(minutes=min_buffer))[0]
        j_ran = np.arange(j_sta, j_end, 1/86400)[:-1]
        
        d = [*set(tle_df['id'])] # all sat norads
        e = tle_df.set_index('id')
        
        # observation satellite
        obs_e = e.loc[obs]['epoch']
        obs_i = np.argmin([abs(j.total_seconds()) for j in (obs_e - i)])
        obs_s = e.loc[obs].iloc[obs_i]['tle_1'][0]
        obs_t = e.loc[obs].iloc[obs_i]['tle_2'][0]

        d.remove(obs)

        for j in d:
            f = e.loc[j]['epoch']
            g = np.argmin([abs(j.total_seconds()) for j in (f - i)])
            s = e.loc[j].iloc[g]['tle_1'][0]
            t = e.loc[j].iloc[g]['tle_2'][0]

            h = Satrec.twoline2rv(s, t).sgp4_array(j_ran, np.zeros(len(j_ran)))[1]
            k = Satrec.twoline2rv(obs_s, obs_t).sgp4_array(j_ran, np.zeros(len(j_ran)))[1]     
            is_dist = np.sum( (h - k)**2, axis=1 )**0.5

            is_mindist = np.min(is_dist)

            if is_mindist < is_dist_buffer:
                is_mindist_list.append(is_mindist)
                sec_diff = int(np.argmin(is_dist) - int(len(j_ran)/2))
                is_mindist_t.append(i + timedelta(seconds = sec_diff))
                is_mindist_id.append(j)
                alt = np.sum(k[np.argmin(is_dist)]**2)**0.5 - r_e
                is_mindist_alt.append(alt)
#                 print('--', j)
            else:
                continue
        
    return is_mindist_id, is_mindist_t, is_mindist_list, is_mindist_alt

In [5]:
%%time
tle_get(datetime(2011,12,1), 31, '2011-12-01', '2012-01-01', 1500).to_pickle('tle_2011_12.pkl')

Snoozing for 90 secs for rate limit reasons (max 30/min and 300/hr)...
Snoozing for 90 secs for rate limit reasons (max 30/min and 300/hr)...
Snoozing for 90 secs for rate limit reasons (max 30/min and 300/hr)...
CPU times: total: 6.7 s
Wall time: 5min 27s


In [35]:
%%time
is_id, is_dt, is_dist, is_alt = find_conj(datetime(2011,12,1), 3, str(26464), 10, 500, peri_dt_list)

CPU times: total: 3min 53s
Wall time: 3min 56s


In [37]:
pd.DataFrame({'id': is_id, 'time': is_dt, 'dist_km': is_dist, 'alt_km': is_alt}).to_pickle('2011_12_c2_conj.pkl')

In [43]:
c1 = pd.read_pickle('2011_07_c2_conj.pkl')
c2 = pd.read_pickle('2011_08_c2_conj.pkl')
c3 = pd.read_pickle('2011_09_c2_conj.pkl')
c4 = pd.read_pickle('2011_10_c2_conj.pkl')
c5 = pd.read_pickle('2011_11_c2_conj.pkl')
c6 = pd.read_pickle('2011_12_c2_conj.pkl')

pd.concat([c1, c2, c3, c4, c5, c6]).sort_values(by='time').reset_index(drop=True).to_pickle('all_conj_list.pkl')